In [1]:
from var_func import *

## Matriks Koefiisien Input

In [2]:
def get_coef_matrix(by, aggregate):
    '''
    aggregate : ['input', 'output']
    '''
    row_col = {
        'industri':52, 'pulau':6, 'lapus':17, 'provinsi':34, 'provinsi_industri':34*52, 
        'provinsi_lapus':34*17, 'pulau_industri':6*52, 'pulau_lapus':6*17
    }
    
    if aggregate not in ['input', 'output']:
        raise Exception("Argumen aggregate harus diantara nilai berikut : ['input', 'output']")
    
    if by not in row_col.keys():
        raise Exception("Argumen by harus diantara nilai berikut : ['industri', 'pulau', 'lapus', 'provinsi', 'industri_provinsi', 'industri_pulau', 'lapus_provinsi', 'lapus_pulau']")
    
    
    table_name,nilai,shape = (tables[f'ki_{by}']['name'],tables[f'ki_{by}']['attr']['nilai'],row_col[by]) if aggregate == 'input' else (tables[f'ko_{by}']['name'],tables[f'ko_{by}']['attr']['nilai'],row_col[by])
    
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(f'''SELECT {nilai} FROM {table_name}''')
    mat = np.array(cur.fetchall()).reshape(shape,shape)
    cur.close()
    conn.close()
    return mat

## Input Output yang digunakan antar provinsi/industri

## Aggregate

In [3]:
def change_code(data, columns_reference):
    '''
    data : dataframe
    columns_reference : dict. Contoh {'kode_provinsi':'provinsi'}
    '''
    data_copied = data.copy()
    for column, reference in columns_reference.items():
        data_copied = pd.concat([
            data_copied.set_index(column),
            tables[f'kode_{reference}']['data'].set_index(tables[f'kode_{reference}']['attr']['kode'])
        ], axis = 1, join = "inner").rename(columns = {tables[f'kode_{reference}']['attr']['nama']:column})
    
    return data_copied.reset_index(drop = True)

In [4]:
def io_used(by, condition, aggregate):
    if by not in ['industri','pulau','provinsi','lapus','provinsi_industri','provinsi_lapus','pulau_industri','pulau_lapus']:
        raise Exception('salah nilai untuk argumen by')
    if type(condition) is not dict:
        raise Exception('condition harus bertipe dict')
    if not (set(by.split('_')) <= set(condition.keys())):
        raise Exception(f'key pada condition harus memiliki unsur dari {set(by.split("_"))}')
    if aggregate not in ['input', 'output']:
        raise Exception(f'Nilai aggregate harus diantara ["input","output"]')
        
    conn = get_connection()
    
    table_name = tables[f'ko_{by}']['name'] if aggregate == 'output' else tables[f'ki_{by}']['name']
    sql_condition = ' AND '.join([
        f'{tables[table_name]["attr"][f"{i}_penyedia"]} = %({i})s' for i in condition.keys()
    ]) if aggregate == 'output' else ' AND '.join([
        f'{tables[table_name]["attr"][f"{i}_pengguna"]} = %({i})s' for i in condition.keys()
    ])

    sql = f'''
    SELECT * FROM {table_name}
    WHERE {sql_condition}
    ORDER BY {tables[table_name]['attr']['nilai']} DESC
    '''
    
    data = pd.read_sql(sql, conn, params = condition)
    conn.close()
    return data
    

### Test Function

In [41]:
# import random 

# for by in ['industri','pulau','provinsi','lapus','provinsi_industri','provinsi_lapus','pulau_industri','pulau_lapus']:
#     condition = {}
#     for key in by.split('_'):
#         condition[key] = random.randint(0,100)
        
#     for aggregate in ['input','output']:
#         print(
#             io_used(
#                 by,
#                 condition,
#                 aggregate
#             )
#         )

change_code(
    io_used('provinsi_lapus', {'provinsi':'94','lapus':'B'},'output'),
    {
        'provinsi_penyedia':'provinsi', 'provinsi_pengguna':'provinsi',
        'lapangan_usaha_penyedia':'lapus', 'lapangan_usaha_pengguna':'lapus'
    }
)

,nilai,provinsi_penyedia,provinsi_pengguna,lapangan_usaha_penyedia,lapangan_usaha_pengguna
0,1.733637e-01,Papua,Papua,Pertambangan dan Penggalian,Pertambangan dan Penggalian
1,1.562570e-01,Papua,Jawa Timur,Pertambangan dan Penggalian,Industri Pengolahan
2,1.066044e-01,Papua,Jawa Barat,Pertambangan dan Penggalian,Industri Pengolahan
3,5.638960e-02,Papua,DKI Jakarta,Pertambangan dan Penggalian,Industri Pengolahan
4,2.508984e-02,Papua,Papua,Pertambangan dan Penggalian,Konstruksi
...,...,...,...,...,...
573,1.483286e-17,Papua,Nusa Tenggara Timur,Pertambangan dan Penggalian,Pertambangan dan Penggalian
574,1.398441e-17,Papua,Nusa Tenggara Timur,Pertambangan dan Penggalian,Jasa Keuangan dan Asuransi
575,9.658178e-18,Papua,Nusa Tenggara Timur,Pertambangan dan Penggalian,Real Estate
576,7.869297e-18,Papua,Nusa Tenggara Timur,Pertambangan dan Penggalian,Jasa Perusahaan


## Dari Provinsi - Industri

In [89]:
def sql_total_io(unit):
    total_io = tables["total_io"]["name"]
    nilai_juta = tables["total_io"]["attr"]["nilai_juta"]

    if unit == 'provinsi_industri':
        return f'SELECT * FROM {total_io}'
    elif unit in ['industri', 'provinsi']:
        return f'''
        SELECT 
        {unit},
        sum({nilai_juta}) as nilai_juta
        FROM {total_io}
        GROUP BY {unit}
        '''
    elif unit in ['pulau','lapus']:
        rel_table = tables['rel_lapus_industri']['name'] if unit == 'lapus' else tables['rel_pulau_provinsi']['name']
        forkey_column = tables['total_io']['attr']['industri'] if unit == 'lapus' else tables['total_io']['attr']['provinsi']
        refer_column = tables['rel_lapus_industri']['attr']['kode_industri'] if unit == 'lapus' else tables['rel_pulau_provinsi']['attr']['kode_provinsi']
        show_refer_column, show_column = (tables['rel_lapus_industri']['attr']['kode_lapus'], 'lapangan_usaha') if unit == 'lapus' else (tables['rel_pulau_provinsi']['attr']['kode_pulau'], 'pulau')

        return f'''
        SELECT 
        b.{show_refer_column} AS {show_column},
        SUM(a.{nilai_juta}) AS nilai_juta
        FROM {total_io} a
        LEFT JOIN {rel_table} b
        ON b.{refer_column} = a.{forkey_column}
        GROUP BY b.{show_refer_column}
        '''

    elif unit in ['provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
        column_name = {'provinsi':'provinsi', 'industri':'industri', 'pulau':'pulau', 'lapus':'lapangan_usaha'}
        column_selected, table_joined = [], ['']

        for i in unit.split('_'):
            if i == 'pulau':
                column_selected.append(f'b1.{tables["rel_pulau_provinsi"]["attr"]["kode_pulau"]} AS pulau')
                table_joined.append(f'''
                {tables["rel_pulau_provinsi"]["name"]} AS b1
                ON b1.{tables["rel_pulau_provinsi"]["attr"]["kode_provinsi"]} = a.{tables["total_io"]["attr"]["provinsi"]}
                ''')
            if i == 'lapus':
                column_selected.append(f'b2.{tables["rel_lapus_industri"]["attr"]["kode_lapus"]} AS lapangan_usaha')
                table_joined.append(f'''
                {tables["rel_lapus_industri"]["name"]} AS b2
                ON b2.{tables["rel_lapus_industri"]["attr"]["kode_industri"]} = a.{tables["total_io"]["attr"]["industri"]}
                ''')
            if i not in ['pulau','lapus']:
                column_selected.append(f'a.{tables["total_io"]["attr"][i]}')
        column_selected.append(f'SUM(a.{nilai_juta}) AS nilai_juta')

        return f'''
        SELECT 
        {", ".join(column_selected)}
        FROM {total_io} AS a
        {' LEFT JOIN '.join(table_joined)}
        GROUP BY 
        {", ".join([column_name[i] for i in unit.split('_')])}
        '''

In [90]:
def sql_ia_per_unit(from_, to_):
    
    if from_ == to_:
        return f'SELECT * FROM {tables[f"input_antara_nasional_{from_}"]["name"]}'
    else:
        column_name = {'provinsi':'provinsi', 'industri':'industri', 'pulau':'pulau', 'lapus':'lapangan_usaha'}
        column_selected = []
        table_joined = ['']
        if from_ in ['industri','provinsi']:
            column_selected.append(f'a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{from_}_penyedia"]}')
            
            if to_ in ['industri', 'provinsi']:
                column_selected.append(f'a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{to_}_pengguna"]}')
        
            elif to_ in ['pulau', 'lapus']:
                column_selected.append(f'''{'b1' if to_ == 'pulau' else 'b2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{to_}"]} AS {'pulau' if to_ == 'pulau' else 'lapangan_usaha'}_pengguna''')
                table_joined.append(
                    f'''
                    {tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["name"]} AS {'b1' if to_ == 'pulau' else 'b2'}
                    ON {'b1' if to_ == 'pulau' else 'b2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if to_ == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if to_ == 'pulau' else 'industri'}_pengguna"]}
                    '''
                )
            elif to_ in ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
                for unit in to_.split('_'):
                    if unit in ['pulau', 'lapus']:
                        column_selected.append(f'''{'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{unit}"]} AS {column_name[unit]}_pengguna''')
                        table_joined.append(
                            f'''
                            {tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["name"]} AS {'c1' if unit == 'pulau' else 'c2'}
                            ON {'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if unit == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if unit == 'pulau' else 'industri'}_pengguna"]}
                            '''
                        )
                    else:
                        column_selected.append(f'a.{unit}_pengguna')
                
        elif from_ in ['pulau', 'lapus']:
            column_selected.append(f'''{'b1' if from_ == 'pulau' else 'b2'}.{tables[f"rel_{from_}_{'provinsi' if from_ == 'pulau' else 'industri'}"]["attr"][f"kode_{from_}"]} AS {'pulau' if from_ == 'pulau' else 'lapangan_usaha'}_penyedia''')
            table_joined.append(
                f'''
                {tables[f"rel_{from_}_{'provinsi' if from_ == 'pulau' else 'industri'}"]["name"]} AS {'b1' if from_ == 'pulau' else 'b2'}
                ON {'b1' if from_ == 'pulau' else 'b2'}.{tables[f"rel_{from_}_{'provinsi' if from_ == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if from_ == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if from_ == 'pulau' else 'industri'}_penyedia"]}
                '''
            )
            if to_ in ['industri', 'provinsi']:
                column_selected.append(f'a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{to_}_pengguna"]}')
            elif to_ in ['pulau', 'lapus']:
                column_selected.append(f'''{'b1' if to_ == 'pulau' else 'b2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{to_}"]} AS {'pulau' if to_ == 'pulau' else 'lapangan_usaha'}_pengguna''')
                table_joined.append(
                    f'''
                    {tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["name"]} AS {'b1' if to_ == 'pulau' else 'b2'}
                    ON {'b1' if to_ == 'pulau' else 'b2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if to_ == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if to_ == 'pulau' else 'industri'}_pengguna"]}
                    '''
                )
            elif to_ in ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
                for unit in to_.split('_'):
                    if unit in ['pulau', 'lapus']:
                        column_selected.append(f'''{'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{unit}"]} AS {column_name[unit]}_pengguna''')
                        table_joined.append(
                            f'''
                            {tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["name"]} AS {'c1' if unit == 'pulau' else 'c2'}
                            ON {'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if unit == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if unit == 'pulau' else 'industri'}_pengguna"]}
                            '''
                        )
                    else:
                        column_selected.append(f'a.{unit}_pengguna')
                
        elif from_ in ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
            for unit in from_.split('_'):
                if unit in ['pulau', 'lapus']:
                    column_selected.append(f'''{'b1' if unit == 'pulau' else 'b2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{unit}"]} AS {column_name[unit]}_penyedia''')
                    table_joined.append(
                        f'''
                        {tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["name"]} AS {'b1' if unit == 'pulau' else 'b2'}
                        ON {'b1' if unit == 'pulau' else 'b2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if unit == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if unit == 'pulau' else 'industri'}_penyedia"]}
                        '''
                    )
                else:
                    column_selected.append(f'a.{unit}_penyedia')
                
            if to_ in ['industri','provinsi']:
                column_selected.append(f'a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{to_}_pengguna"]}')
            elif to_ in ['pulau', 'lapus']:
                column_selected.append(f'''{'c1' if to_ == 'pulau' else 'c2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{to_}"]} AS {'pulau' if to_ == 'pulau' else 'lapangan_usaha'}_pengguna''')
                table_joined.append(
                    f'''
                    {tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["name"]} AS {'c1' if to_ == 'pulau' else 'c2'}
                    ON {'c1' if to_ == 'pulau' else 'c2'}.{tables[f"rel_{to_}_{'provinsi' if to_ == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if to_ == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if to_ == 'pulau' else 'industri'}_pengguna"]}
                    '''
                )
            elif to_ in ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
                for unit in to_.split('_'):
                    if unit in ['pulau', 'lapus']:
                        column_selected.append(f'''{'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{unit}"]} AS {column_name[unit]}_pengguna''')
                        table_joined.append(
                            f'''
                            {tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["name"]} AS {'c1' if unit == 'pulau' else 'c2'}
                            ON {'c1' if unit == 'pulau' else 'c2'}.{tables[f"rel_{unit}_{'provinsi' if unit == 'pulau' else 'industri'}"]["attr"][f"kode_{'provinsi' if unit == 'pulau' else 'industri'}"]} = a.{tables["input_antara_nasional_provinsi_industri"]["attr"][f"{'provinsi' if unit == 'pulau' else 'industri'}_pengguna"]}
                            '''
                        )
                    else:
                        column_selected.append(f'a.{unit}_pengguna')
        
        groupby = [f"{column_name[i]}_penyedia" for i in from_.split('_')] + [f"{column_name[j]}_pengguna" for j in to_.split('_')]
        column_selected.append(f'SUM(a.{tables["input_antara_nasional_provinsi_industri"]["attr"]["nilai_juta"]}) AS nilai_juta')
        return f'''SELECT 
        {', '.join(column_selected)}
        FROM {tables["input_antara_nasional_provinsi_industri"]["name"]} a
        {' LEFT JOIN '.join(table_joined)}
        GROUP BY 
        {", ".join(groupby)}'''

### Testing

In [91]:
# conn = get_connection()
# x = 0
# for i in ['industri', 'provinsi', 'lapus', 'pulau'] + ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
#     for j in ['industri', 'provinsi', 'lapus', 'pulau'] + ['provinsi_industri', 'provinsi_lapus', 'pulau_industri', 'pulau_lapus']:
#         x=x+1
#         if x < 41:
#             continue
#         print(f'\n\nFrom {i} To {j}')
#         print(pd.read_sql(sql_ia_per_unit(i,j), conn))

In [92]:
conn.close()

In [ ]:
def sql_ia_where(from_cond, to_cond):
    if set(from_cond.keys()) <= {'industri','pulau','provinsi','lapus'}:
        raise Exception('1')
    if set(to_cond.keys()) <= {'industri','pulau','provinsi','lapus'}:
        raise Exception('2')
    
    relation = {
        'provinsi':'rel_pulau_provinsi',
        'pulau':'rel_pulau_provinsi',
        'industri':'rel_lapus_industri',
        'lapus':'rel_lapus_industri'
    }
    
    condition = []
    subquery = []
    i = 0
    for key, item in from_cond.items():
        subquery[i] = f'''
        SELECT
        '''

In [100]:
def io_used_per_unit(from_, to_, aggregate, condition):
    if from_ not in ['industri','pulau','provinsi','lapus','provinsi_industri','provinsi_lapus','pulau_industri','pulau_lapus']:
        raise Exception('1')
    
    if to_ not in ['industri','pulau','provinsi','lapus','provinsi_industri','provinsi_lapus','pulau_industri','pulau_lapus']:
        raise Exception('2')
    
    if (type(from_) is not str) or (type(to_) is not str):
        raise Exception('3')
    
#     if not ((set(from_.split('_')) <= set(condition.keys())) and (set(to_.split('_')) <= set(condition.keys()))):
#         raise Exception('4')
    
    if aggregate not in ['input','output']:
        raise Exception('5')
    column_name = {'provinsi':'provinsi', 'industri':'industri', 'pulau':'pulau', 'lapus':'lapangan_usaha'}
    column_selected = []
    on_ = [
        f'a.{column_name[from_unit]}_penyedia = b.{column_name[from_unit]}' for from_unit in from_.split('_')
    ] if aggregate == 'output' else [
        f'a.{column_name[to_unit]}_pengguna = b.{column_name[to_unit]}' for to_unit in to_.split('_')
    ]
    
    for from_unit in from_.split('_'):
        column_selected.append(f'a.{column_name[from_unit]}_penyedia')
    for to_unit in to_.split('_'):
        column_selected.append(f'a.{column_name[to_unit]}_pengguna')
    
    column_selected.append(f'a.nilai_juta/b.nilai_juta as nilai')
    
    return f'''
    WITH c AS (
        WITH b AS (
            {sql_total_io(from_) if aggregate == 'output' else sql_total_io(to_)}
        ), a AS (
            {sql_ia_per_unit(from_,to_)}
        )
        SELECT 
        {', '.join(column_selected)}
        FROM a
        LEFT JOIN b
        ON {' AND '.join(on_)}
    )
    SELECT * FROM c
    WHERE {' AND '.join([f'{key} = %({key})s' for key in condition.keys()])}
    ORDER BY nilai DESC
    '''
    

### Testing

In [38]:
# a = ['industri','pulau','provinsi','lapus','provinsi_industri','provinsi_lapus','pulau_industri','pulau_lapus']
# d = {}
# for k in a:
#     d[k] = 'cc'
# conn = get_connection()

# for i in a:
#     for j in a:
#         for x in ['input','output']:
#             start = timer()
#             print(f'{i} to {j} with {x}')
#             print(pd.read_sql(io_used_per_unit(i, j, x, d), conn).head())
#             print(timer() - start)

# conn.close()

# Dash App

In [51]:
from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [97]:
# Load Data
prov_options = []
ind_options = []
for i in tables['kode_provinsi']['data'].values:
    prov_options.append({'label':f'{i[1]}', 'value':f'{i[0]}'})

for i in tables['kode_industri']['data'].values:
    ind_options.append({'label':f'{i[1]}', 'value':f'{i[0]}'})

agg_options = [
    {'label':'Input yang digunakan dari industri/provinsi lain', 'value':'input'},
    {'label':'Output yang digunakan oleh industri/provinsi lain', 'value':'output'}
]
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Label([
        "Pilih Jenis Agregasi",
        dcc.RadioItems(
            id = 'agg-radio',
            value='output', options = agg_options
        )
    ]),
    html.Div([
        html.Label([
            "Pilih Provinsi",
            dcc.Dropdown(
                id='prov-dropdown', clearable=False,
                value='11', options=prov_options)
        ])
    ],style = {'width':'49%', 'display':'inline-block'}), 
    html.Div([
        html.Label([
            "Pilih Industri",
            dcc.Dropdown(
                id='ind-dropdown', clearable=False,
                value='I-01', options=ind_options)
        ])
    ], style = {'width':'49%', 'display':'inline-block'}),
    html.H1(id="agg-title"),
    html.H2(id='ind-prov-title'),
    dcc.Graph(id='graph'),
])
# Define callback to update graph
@app.callback(
    Output('agg-title', 'children'),
    Output('ind-prov-title', 'children'),
    Output('graph', 'figure'),
    Input("prov-dropdown", "value"),
    Input("ind-dropdown", "value"),
    Input("agg-radio", "value")
)
def update_figure(prov_code, ind_code, agg_type):
    kode_provinsi = tables['kode_provinsi']['data']
    kode_industri = tables['kode_industri']['data']
    provinsi = kode_provinsi[kode_provinsi[tables['kode_provinsi']['attr']['kode']] == prov_code][tables['kode_provinsi']['attr']['nama']].reset_index(drop = True)[0]
    industri = kode_industri[kode_industri[tables['kode_industri']['attr']['kode']] == ind_code][tables['kode_industri']['attr']['nama']].reset_index(drop = True)[0]

    coef_table_name = 'ki_provinsi_industri' if agg_type == 'input' else 'ko_provinsi_industri'
    a = 'pengguna' if agg_type == 'output' else 'penyedia'
    h1_title = 'Output yang digunakan oleh Industri/Provinsi lain dari' if agg_type == 'output' else 'Input yang digunakan dari Industri/Provinsi lain oleh'
    h2_title = f'{provinsi} - {industri}'
    data = change_code(
        io_used('provinsi_industri',{'provinsi':prov_code,'industri':ind_code},agg_type).head(15),
        {
            tables[coef_table_name]['attr'][f'provinsi_{a}']:'provinsi',
            tables[coef_table_name]['attr'][f'industri_{a}']:'industri'
        }
    ).sort_values(tables[coef_table_name]['attr']['nilai'])
    data['prov_ind'] = data[tables[coef_table_name]['attr'][f'provinsi_{a}']] + ' - ' + data[tables[coef_table_name]['attr'][f'industri_{a}']]
    
    fig = px.bar(
        data, 
        x = tables[coef_table_name]['attr']['nilai'], 
        y = 'prov_ind',
        custom_data = [
            data[tables[coef_table_name]['attr']['nilai']] * 100,
            data[tables[coef_table_name]['attr'][f'provinsi_{a}']],
            data[tables[coef_table_name]['attr'][f'industri_{a}']],
        ]
    )
    
    fig.update_traces(
        hovertemplate = '<br>'.join([
            'Provinsi = %{customdata[1]}',
            'Industri = %{customdata[2]}',
            'Output = %{customdata[0]:.2f}%' if agg_type == 'output' else 'Input = %{customdata[0]:.1f}%'
        ])
    )
    
    return h1_title,h2_title,fig
# Run app and display result inline in the notebook
app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/


In [13]:
total_io = pd.read_sql('SELECT industri, sum(nilai_juta) as nilai_juta FROM total_io group by industri', conn)
total_input_antara_nasional = pd.read_sql('SELECT industri_pengguna, sum(nilai_juta) as nilai_juta from total_input_antara_nasional group by industri_pengguna', conn)
total_input_antara = pd.read_sql('SELECT industri_pengguna, sum(nilai_juta) as nilai_juta FROM total_input_antara GROUP BY industri_pengguna', conn)

In [15]:
a = total_io.copy()
a['rasio input_antara_nasional'] = total_input_antara_nasional.sort_values('industri_pengguna')['nilai_juta']/total_io.sort_values('industri')['nilai_juta']
a['rasio input_antara'] = total_input_antara.sort_values('industri_pengguna')['nilai_juta']/total_io.sort_values('industri')['nilai_juta']

In [16]:
a.sort_values('rasio input_antara')

,industri,nilai_juta,rasio input_antara_nasional,rasio input_antara
5,I-06,1.132271e+08,0.122630,0.130260
6,I-07,3.256530e+08,0.141557,0.148448
1,I-02,2.588704e+08,0.165817,0.180984
0,I-01,3.552091e+08,0.175688,0.192079
2,I-03,4.904935e+08,0.210574,0.219099
46,I-47,7.560396e+08,0.204771,0.226736
4,I-05,2.430300e+07,0.212786,0.229236
42,I-43,4.527860e+08,0.210748,0.244023
7,I-08,3.993900e+08,0.218178,0.250868
13,I-14,3.787298e+08,0.240292,0.265861


In [109]:
# input_coef = pd.read_sql('SELECT * FROM ki_industri_provinsi ORDER BY provinsi_penyedia, industri_penyedia, provinsi_pengguna, industri_pengguna', conn)
start = timer()
input_coef = pd.read_sql('SELECT * FROM ki_industri_provinsi', conn)
timer() - start

924.5911662430008

In [77]:
input_coef.loc[1768*4]

,provinsi_penyedia,industri_penyedia,provinsi_pengguna,industri_pengguna,nilai
0,11,I-01,11,I-01,0.045934
1,11,I-01,11,I-02,0.000000
2,11,I-01,11,I-03,0.000000
3,11,I-01,11,I-04,0.017800
4,11,I-01,11,I-05,0.000000
...,...,...,...,...,...
3125819,94,I-52,94,I-48,0.000076
3125820,94,I-52,94,I-49,0.001557
3125821,94,I-52,94,I-50,0.002031
3125822,94,I-52,94,I-51,0.002011


In [79]:
cur = conn.cursor()

In [123]:
start = timer()
cur.execute('SELECT nilai FROM ki_industri_provinsi ORDER BY provinsi_penyedia, industri_penyedia, provinsi_pengguna, industri_pengguna')
# cur.execute('SELECT nilai FROM ki_industri_provinsi')
timer() - start

60.23776516300131

In [115]:
start = timer()
a = cur.fetchall()
timer() - start

2.554928215999098

In [3]:
conn = get_connection()
data = pd.read_sql('SELECT * FROM input_antara_nasional', conn)
conn.close()

In [26]:
(pd.concat([
    pd.concat([
        data.set_index('industri_pengguna'),
        tables['rel_lapus_industri']['data'].set_index('kode_industri')
    ], axis = 1, join = "inner")
     .reset_index()
     .rename(columns = {'index':'industri_pengguna', 'kode_lapus':'lapus_pengguna'})
     .set_index('industri_penyedia'),
    tables['rel_lapus_industri']['data'].set_index('kode_industri')
    ], axis = 1, join = "inner")
 .reset_index()
 .rename(columns = {'index':'industri_penyedia', 'kode_lapus':'lapus_penyedia'})
).groupby(['lapus_penyedia', 'lapus_pengguna']).sum()['nilai_juta'].reset_index().sort_values(['lapus_penyedia', 'lapus_pengguna'])

,lapus_penyedia,lapus_pengguna,nilai_juta
0,A,A,1.243607e+08
1,A,B,1.402522e+06
2,A,C,8.115614e+08
3,A,D,2.119301e+04
4,A,E,3.042085e+04
...,...,...,...
284,RSTU,MN,6.184203e+06
285,RSTU,O,5.535028e+06
286,RSTU,P,6.833625e+06
287,RSTU,Q,2.584298e+06


In [25]:
data.groupby(['provinsi_pengguna', 'provinsi_penyedia'])[['nilai_juta']].sum().reset_index().sort_values(['provinsi_penyedia', 'provinsi_pengguna'])

,provinsi_pengguna,provinsi_penyedia,nilai_juta
0,11,11,6.796316e+07
34,12,11,1.300372e+07
68,13,11,7.113993e+05
102,14,11,3.383422e+05
136,15,11,1.749701e+05
...,...,...,...
1019,76,94,1.891818e+04
1053,81,94,6.554274e+04
1087,82,94,3.736250e+04
1121,91,94,1.868153e+05


In [23]:
conn = get_connection()
data2 = pd.read_sql('SELECT * FROM input_antara_nasional_provinsi', conn)
conn.close()

In [24]:
data2

,provinsi_penyedia,provinsi_pengguna,nilai_juta
0,11,11,6.796316e+07
1,11,12,1.300372e+07
2,11,13,7.113993e+05
3,11,14,3.383422e+05
4,11,15,1.749701e+05
...,...,...,...
1151,94,76,1.891818e+04
1152,94,81,6.554274e+04
1153,94,82,3.736250e+04
1154,94,91,1.868153e+05
